In [1]:
from hydraseq import *
from collections import defaultdict

In [2]:
sentence = "spring leaves spring"

hdr0 = hydraseq.Hydraseq('0_')
hdr1 = hydraseq.Hydraseq('1_')
hdr2 = hydraseq.Hydraseq('2_')

for pattern in [
    "spring 0_A_",
    "spring 0_V_",
    "leaves 0_N_",
    "leaves 0_V_",
    "fall 0_A_",
    "fall 0_V_"
]:
    hdr0.insert(pattern)

for pattern in [
    "0_N_ 1_NP_",
    "0_A_ 0_N_ 1_NP_",
    "0_V_ 1_VP_",
    "0_V_ 0_N_ 1_VP_"
]:
    hdr1.insert(pattern)

for pattern in [
    "1_NP_ 1_VP_ 2_S_"
]:
    hdr2.insert(pattern)


In [3]:
hdr = Hydraseq("-")
hdr.insert("spring leaves spring _sentence")
hydras = [hdr, hdr0, hdr1, hdr2]
think(hydras)

START CONVO  [['spring'], ['leaves'], ['spring']]   0_
WORD= ['spring'] depth= 0 idx= 1  ACTIVE SEQ:  [['spring']]  next_word= 0_A_
WORD= ['spring'] depth= 0 idx= 1  ACTIVE SEQ:  [['spring']]  next_word= 0_V_
WORD= ['leaves'] depth= 1 idx= 2  ACTIVE SEQ:  [['leaves']]  next_word= 0_N_
WORD= ['leaves'] depth= 1 idx= 2  ACTIVE SEQ:  [['leaves']]  next_word= 0_V_
WORD= ['spring'] depth= 2 idx= 3  ACTIVE SEQ:  [['spring']]  next_word= 0_A_
WORD= ['spring'] depth= 2 idx= 3  ACTIVE SEQ:  [['spring']]  next_word= 0_V_
START CONVO  [['0_A_', '0_V_'], ['0_N_', '0_V_'], ['0_A_', '0_V_']]   1_
WORD= ['0_A_', '0_V_'] depth= 0 idx= 1  ACTIVE SEQ:  [['0_V_']]  next_word= 1_VP_
WORD= ['0_N_', '0_V_'] depth= 0 idx= 2  ACTIVE SEQ:  [['0_A_', '0_N_']]  next_word= 1_NP_
WORD= ['0_N_', '0_V_'] depth= 0 idx= 2  ACTIVE SEQ:  [['0_V_', '0_N_']]  next_word= 1_VP_
WORD= ['0_N_', '0_V_'] depth= 1 idx= 2  ACTIVE SEQ:  [['0_N_']]  next_word= 1_NP_
WORD= ['0_N_', '0_V_'] depth= 1 idx= 2  ACTIVE SEQ:  [['0_V_']]  n

[[[[0, 1, ['spring']], [1, 2, ['leaves']], [2, 3, ['spring']]]],
 [[[0, 1, ['0_A_', '0_V_']],
   [1, 2, ['0_N_', '0_V_']],
   [2, 3, ['0_A_', '0_V_']]]],
 [[[0, 2, ['1_NP_', '1_VP_']], [2, 3, ['1_VP_']]],
  [[0, 1, ['1_VP_']], [1, 2, ['1_NP_', '1_VP_']], [2, 3, ['1_VP_']]]],
 [[[0, 2, ['2_S_']]], [[1, 3, ['2_S_']]]]]

In [4]:
from collections import namedtuple
VertNode = namedtuple("VertNode", ["sentence", "lasts", "nexts"])

In [5]:
class VertiNode:
    def __init__(self, start, end, word):
        self.uppers = []
        self.lowers = []
        self.start = start
        self.end = end
        self.word = word
        self.key = "{}_{}
        
        
{}".format(self.start, self.word, self.end)

    def __repr__(self):
        return "<key={},up={},down={}>".format(self.key, len(self.uppers), len(self.lowers))
Th = {} 

def node(start, end, word):
    node =  Th.get("{}_{}_{}".format(start,word,end), None)
    if node:
        return node
    else:
        node = VertiNode(start, end, word)
        Th[node.key] = node
        return node
        

def to_nodes(lst_lst_words):
    return [[VertiNode(word) for word in lst_words] for lst_words in lst_lst_words]

def to_words(lst_nodes):
    return [node.word for node in lst_nodes]
sentence = [['spring'],['leaves'],['spring']]

In [6]:
def run_convolutions2(words, hydra, debug=False):
    """Run convolution on words using the hydra provided.
    Args:
        words, list<list<strings>>, the words, usually representing a coherent sentence or phrase
        hydra, hydraseq, a trained hydra usually trained on the set of words used in sentence.
        debug, output intermediate steps to console
    Returns:
        a list of convolutions, where each convolution is [start, end, [words]]
    """
    print("START CONVO ", words, " ", hydra.uuid)
    words = words if isinstance(words, list) else hydra.get_word_array(words)
    [[node(idx, subword, idx+1)] for idx, word in enumerate(words) for subword in word]
    if debug: print(words)
    hydras = []
    results = []

    for idx, word in enumerate(words):
        if debug: print(word)
        word_results = []
        hydras.append(Hydraseq(idx, hydra))
        for depth, _hydra in enumerate(hydras):
            next_hits = []
            for next_word in _hydra.hit(word, is_learning=False).get_next_values():
                if next_word.startswith(hydra.uuid):
                    next_hits.append(next_word)
                    print(
                        "WORD=",word,
                        "depth=",depth,
                        "idx=",idx+1,
                        " ACTIVE SEQ: ",[sequ.split()[1:-1] for sequ in _hydra.get_next_sequences() if sequ.split()[-1] == next_word],
                        " next_word=", next_word
                        )
                    for subword in word:
                        node(depth, next_word, idx+1).lowers.append(node(depth, subword, idx+1))
                        node(depth, subword, idx+1).uppers.append(node(depth, next_word, idx+1))
            if debug: print(next_hits)
            if next_hits: word_results.append([depth, idx+1, next_hits])
        results.extend(word_results)
    return results

In [7]:
def BFS(hydras):
    def get_next_sentences(sentence, hydra):
        return reconstruct(to_tree_nodes(run_convolutions2(patterns_only(sentence), hydra)))

    init_node = VertNode(get_init_sentence_from_hydra(hydras[0])[0], [],[])
    print("init_node ", init_node)
    fringe = [init_node]
    ends = [init_node]
    idx = 1
    while fringe:
        if idx >= len(hydras): break
        for node in fringe:
            fringe.remove(node)
            print('\nnode.sentence=', node.sentence, '\npatterns_only=', patterns_only(node.sentence), '\nactive_sequence=', hydras[idx-1].look_ahead(patterns_only(node.sentence)).get_active_sequences())
            fringe.extend([VertNode(sentence, [],[]) for sentence in get_next_sentences(node.sentence, hydras[idx])])
            print('fringe2 ', [node.sentence for node in fringe])
            for f_node in fringe:
                if f_node in ends: ends.remove(f_node)
                ends.append(f_node)
                f_node.lasts.append(node)
        idx +=1
        print("idx=",idx)
    return ends
ends = BFS(hydras)

init_node  VertNode(sentence=[[0, 1, ['spring']], [1, 2, ['leaves']], [2, 3, ['spring']]], lasts=[], nexts=[])

node.sentence= [[0, 1, ['spring']], [1, 2, ['leaves']], [2, 3, ['spring']]] 
patterns_only= [['spring'], ['leaves'], ['spring']] 
active_sequence= ['(*) spring leaves spring']
START CONVO  [['spring'], ['leaves'], ['spring']]   0_
WORD= ['spring'] depth= 0 idx= 1  ACTIVE SEQ:  [['spring']]  next_word= 0_A_
WORD= ['spring'] depth= 0 idx= 1  ACTIVE SEQ:  [['spring']]  next_word= 0_V_
WORD= ['leaves'] depth= 1 idx= 2  ACTIVE SEQ:  [['leaves']]  next_word= 0_N_
WORD= ['leaves'] depth= 1 idx= 2  ACTIVE SEQ:  [['leaves']]  next_word= 0_V_
WORD= ['spring'] depth= 2 idx= 3  ACTIVE SEQ:  [['spring']]  next_word= 0_A_
WORD= ['spring'] depth= 2 idx= 3  ACTIVE SEQ:  [['spring']]  next_word= 0_V_
fringe2  [[[0, 1, ['0_A_', '0_V_']], [1, 2, ['0_N_', '0_V_']], [2, 3, ['0_A_', '0_V_']]]]
idx= 2

node.sentence= [[0, 1, ['0_A_', '0_V_']], [1, 2, ['0_N_', '0_V_']], [2, 3, ['0_A_', '0_V_']]] 
pa

In [8]:
results= run_convolutions2(sentence, hdr0)
results

START CONVO  [['spring'], ['leaves'], ['spring']]   0_
WORD= ['spring'] depth= 0 idx= 1  ACTIVE SEQ:  [['spring']]  next_word= 0_A_
WORD= ['spring'] depth= 0 idx= 1  ACTIVE SEQ:  [['spring']]  next_word= 0_V_
WORD= ['leaves'] depth= 1 idx= 2  ACTIVE SEQ:  [['leaves']]  next_word= 0_N_
WORD= ['leaves'] depth= 1 idx= 2  ACTIVE SEQ:  [['leaves']]  next_word= 0_V_
WORD= ['spring'] depth= 2 idx= 3  ACTIVE SEQ:  [['spring']]  next_word= 0_A_
WORD= ['spring'] depth= 2 idx= 3  ACTIVE SEQ:  [['spring']]  next_word= 0_V_


[[0, 1, ['0_A_', '0_V_']], [1, 2, ['0_N_', '0_V_']], [2, 3, ['0_A_', '0_V_']]]

In [9]:
Th

{'0_1*0_A_': <key=0_1*0_A_,up=0,down=0>,
 '0_1*0_V_': <key=0_1*0_V_,up=0,down=0>,
 '0_1*1_NP_': <key=0_1*1_NP_,up=0,down=0>,
 '0_1*1_VP_': <key=0_1*1_VP_,up=0,down=0>,
 '0_1*spring': <key=0_1*spring,up=1,down=0>,
 '0_2*0_N_': <key=0_2*0_N_,up=1,down=0>,
 '0_2*0_V_': <key=0_2*0_V_,up=1,down=0>,
 '0_2*1_NP_': <key=0_2*1_NP_,up=0,down=0>,
 '0_2*1_VP_': <key=0_2*1_VP_,up=1,down=0>,
 '0_2*2_S_': <key=0_2*2_S_,up=0,down=0>,
 '1_2*0_N_': <key=1_2*0_N_,up=0,down=0>,
 '1_2*0_V_': <key=1_2*0_V_,up=0,down=0>,
 '1_2*1_NP_': <key=1_2*1_NP_,up=0,down=0>,
 '1_2*1_VP_': <key=1_2*1_VP_,up=0,down=0>,
 '1_2*leaves': <key=1_2*leaves,up=1,down=0>,
 '2_3*0_A_': <key=2_3*0_A_,up=0,down=0>,
 '2_3*0_V_': <key=2_3*0_V_,up=0,down=0>,
 '2_3*1_VP_': <key=2_3*1_VP_,up=0,down=0>,
 '2_3*spring': <key=2_3*spring,up=1,down=0>}

In [10]:
Th["0_2_2_S_"].lowers[0].key

KeyError: '0_2_2_S_'